In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp arc

In [ ]:
#| export
import json, os
import numpy as np
import gym
from gym import spaces
from time import sleep
import pygame
from matplotlib import colors
# import copy

In [54]:
#| export

class ARCEnv(gym.Env):
    def __init__(self):
        super(ARCEnv, self).__init__()
        self.index = 0
        self.env = None
        self.dimensions = []
        self.fitness = 0.0
        self.state = []
        self.done = False

        # Render settings
        self.screen_width = 800
        self.screen_height = 600
        self.grid_size = 30
        self.screen = None
        self.isopen = True

        self.cmap = colors.ListedColormap(['#000000', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
                                           '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
        self.norm = colors.Normalize(vmin=0, vmax=9)

    def initialise(self, file_name, properties):
        with open(file_name, 'r') as f:
            data = json.load(f)
        
        for key, value in properties.items():
            setattr(self, key, value)
        
        self.train_data = data['train']
        self.test_data = data['test']
        self.inputs = [x['input'] for x in self.train_data]
        self.outputs = [x.get('output', []) for x in self.train_data]
        self.reset()

    def get_train_array(self, idx):
        return self.train_data[idx]

    def get_input_array(self, idx):
        return self.inputs[idx]

    def get_output_array(self, idx):
        return self.outputs[idx]

    def get_element(self, array, row, col):
        return array[row][col]

    def get_index(self):
        return self.index

    def set_index(self, index):
        self.index = index
        self.reset()

    def add_rows(self, num_rows):
        self.env = np.pad(self.env, ((0, num_rows), (0, 0)), mode='constant', constant_values=0)

    def remove_rows(self, num_rows):
        self.env = self.env[:-num_rows, :]

    def add_columns(self, num_columns):
        self.env = np.pad(self.env, ((0, 0), (0, num_columns)), mode='constant', constant_values=0)

    def remove_columns(self, num_columns):
        self.env = self.env[:, :-num_columns]

    def fitness_function(self):
        output = np.array(self.outputs[self.index])
        
        # First metric: Squared difference in dimensions
        dim_metric = (self.env.shape[0] - output.shape[0]) ** 2 + (self.env.shape[1] - output.shape[1]) ** 2
        
        # Second metric: Squared difference in elements
        element_metric = 0
        if getattr(self, 'fitness_type', 'full') != 'dim_only':
            for i in range(max(self.env.shape[0], output.shape[0])):
                for j in range(max(self.env.shape[1], output.shape[1])):
                    env_val = self.env[i, j] if i < self.env.shape[0] and j < self.env.shape[1] else None
                    output_val = output[i, j] if i < output.shape[0] and j < output.shape[1] else None
                    if env_val is None or output_val is None:
                        element_metric += 25
                    else:
                        element_metric += (env_val - output_val) ** 2

        # Final metric: Sum of the two metrics or dim_metric only
        if getattr(self, 'fitness_type', 'full') == 'dim_only':
            final_metric = dim_metric
        else:
            final_metric = dim_metric + element_metric

        return final_metric

    def step(self, action):
        self.process_dimensions(action)
        if getattr(self, 'fitness_type', 'full') != 'dim_only':
            self.process_remaining_values(action[2:])

        self.fitness = self.fitness_function()
        if self.fitness < 1e-6:
            self.done = True
        
        self.state = self.env.flatten().tolist()
        return self.state, self.fitness, self.done

    def reset(self):
        input_shape = np.array(self.inputs[self.index]).shape
        self.env = np.zeros(input_shape, dtype=int)
        self.fitness = self.fitness_function()
        self.done = False
        self.state = self.env.flatten().tolist()
        return self.state

    def get_env_output_dimensions(self):
        input_shape = np.array(self.env).shape
        output_shape = np.array(self.outputs[self.index]).shape
        return input_shape, output_shape

    def process_dimensions(self, action):
        num_rows, num_cols, *values = action

        if num_rows > 0:
            self.add_rows(num_rows)
        elif num_rows < 0:
            self.remove_rows(abs(num_rows))
        
        if num_cols > 0:
            self.add_columns(num_cols)
        elif num_cols < 0:
            self.remove_columns(abs(num_cols))

    def process_remaining_values(self, values):
        for i, value in enumerate(values):
            row, col = divmod(i, self.env.shape[1])
            if row < self.env.shape[0] and col < self.env.shape[1]:
                self.env[row, col] = value

    def render(self, mode='human'):
        def draw_grid(screen, grid, top_left_x, top_left_y, cell_size):
            for i, row in enumerate(grid):
                for j, value in enumerate(row):
                    color = self.cmap(self.norm(value))[:3]  # Get RGB only, excluding alpha
                    color = tuple(int(c * 255) for c in color)  # Multiply each element by 255
                    pygame.draw.rect(screen, color, (top_left_x + j * cell_size, top_left_y + i * cell_size, cell_size, cell_size))
                    pygame.draw.rect(screen, (255, 255, 255), (top_left_x + j * cell_size, top_left_y + i * cell_size, cell_size, cell_size), 1)
            # Draw a black line around the grid
            pygame.draw.rect(screen, (0, 0, 0), (top_left_x, top_left_y, cell_size * len(grid[0]), cell_size * len(grid)), 2)

        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
            pygame.display.set_caption('ARC Environment')

        self.screen.fill((255, 255, 255))

        input_grid = np.array(self.inputs[self.index])
        output_grid = np.array(self.outputs[self.index])
        env_grid = self.env

        draw_grid(self.screen, input_grid, 50, 50, self.grid_size)
        draw_grid(self.screen, output_grid, 300, 50, self.grid_size)
        draw_grid(self.screen, env_grid, 550, 50, self.grid_size)

        # Load and scale images
        arrow_img = pygame.transform.scale(pygame.image.load('images/arrow.png'), (50, 50))
        equals_img = pygame.transform.scale(pygame.image.load('images/equals.jpg'), (50, 50))
        green_tick_img = pygame.transform.scale(pygame.image.load('images/green_tick.png'), (50, 50))
        red_cross_img = pygame.transform.scale(pygame.image.load('images/red-cross.png'), (50, 50))

        self.screen.blit(arrow_img, (250, 150))
        self.screen.blit(equals_img, (500, 150))

        # Display fitness text with font size 20 and bold
        font = pygame.font.Font(None, 20)
        font.set_bold(True)
        fitness_text = font.render(f"Fitness: {self.fitness:.2f}", True, (0, 0, 0))
        self.screen.blit(fitness_text, (600, 400))

        if self.fitness < 1e-6:
            self.screen.blit(green_tick_img, (600, 500))
        else:
            self.screen.blit(red_cross_img, (600, 500))

        pygame.display.flip()

    def close(self):
        if self.screen is not None:
            pygame.display.quit()
            pygame.quit()
            self.isopen = False



In [55]:
#| gui
# Example usage:
props = {'dir': 'C:\\packages\\arc-prize-2024\\training', 'code':'1_007bbfb7.dat', 'fitness_type': 'dim_only'}
file_path = os.path.join(props['dir'], props['code'])
arc_env = ARCEnv()
arc_env.initialise(file_path, props)
arc_env.render()
#    print(state, fitness, done)
print(arc_env.dimensions)
for i in range(6):
    state, fitness, done = arc_env.step([1, 1, 1, 2, 3, 4, 5, 6, 7])
    # print(state, fitness, done)
    print(arc_env.dimensions, fitness, done)
    arc_env.render()
    pass



[]
[] 50 False
[] 32 False
[] 18 False
[] 8 False
[] 2 False
[] 0 True


In [53]:


# sleep(5)
arc_env.close()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()